To dos
1) Set up script to first initialize database image collection (create image collection and write ID image) and then to loop over the dates in the input image collection
2) Add additional reducers  
- For continuous datasets - 5th, 25th, 50th, 75th, and 95th percentile, mean 
- For categorical datasets - Histogram
2) Set up to handle reducers continuous and/or categorical datasets 
3) Add checks whether image asset already exists  
4) Handle NAs
5) Set export parameters as properties (resolution, continuous vs. categorical, unit reduced)  
6) Right now the exports are run over a specified period of time (one month, one year, etc. — iterated), but may want to assess applying to one image at a time
7) Test with RAP

In [21]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AWtgzh4_g1aUokRqmZvLx0CZqqt9bjfW8bNZ0IYbruXohc5iFcJ0Of7oldg

Successfully saved authorization token.


In [22]:
import os
import datetime
from datetime import date
from datetime import timedelta
import pandas as pd

In [23]:
# !pip install geemap
import ee
import geemap
Map = geemap.Map()

## Define Parameters and Load datasets

In [73]:
# ------------------------------------- Define parameters -----------------------------------------------

# Define time period to export
daysToExport = 30
startDate = datetime.datetime(2022, 1, 1)

# Define whether to initialize new image collection or append to existing image collection
process = 'initialize' 


# -------------------------------- Define input Image Collection ----------------------------------------

# Define input Image Collection
in_ic_path = "GRIDMET/DROUGHT"
in_ic = ee.ImageCollection(in_ic_path).filterDate(startDate, startDate + timedelta(days = daysToExport))

# Define variable to generate database table for
var_name = 'long_term_drought_blend'
in_var_type = 'continuous'


# ------------------------------- Define input Feature Collection ---------------------------------------

in_fc_path = 'projects/dri-apps/assets/blm-admin/BLM_Natl_Grazing_Allotment_Polygons_Simplified_clean'
in_fc = ee.FeatureCollection(in_fc_path)
land_unit = 'blm-natl-grazing-allotment-polygons'

# # Subset by geometry
# geometry = ee.Geometry.Polygon([[[-108.4020, 38.7855], [-108.4020, 39.6080], [-109.1823, 39.6080], [-109.1823, 38.7855]]], None, False);
# in_fc = in_fc.filterBounds(geometry)
# # Specify ID property
# in_fc_id = "ALLOT_ID"

# Use full Feature Collection
in_fc = in_fc
# Specify ID property
in_fc_id = "ALLOT_ID"


# --------------------------------- Define database output path -----------------------------------------
var_name_exp = var_name.replace('_', '-')
out_path = f"projects/dri-apps/assets/blm-database/{var_name_exp}"

### Initialize the EE Image Collection and add ID band

In [74]:
# Apply function to select ID column and convert the ID string to numeric
def generate_id_i(in_fc):

    # Function to select ID band
    def select_id(f):
        return(f.select([in_fc_id]).set(in_fc_id, ee.Number.parse(ee.Feature(f.get(in_fc_id)))))
    in_fc = in_fc.map(select_id)

    # Convert feature collection to list
    in_fc_list = in_fc.toList(in_fc.size())

    # Get size of in_fc_list
    in_fc_size = in_fc_list.size()

    # Function to create feature collection next to equator
    def pts_to_equator(i):

        # Cast number to EE number
        i = ee.Number(i)
        
        # Get properties
        properties = ee.Feature(in_fc_list.get(i)).toDictionary()
        
        # Create geometry at equator
        geom = ee.Geometry.Point([i.multiply(0.0002), 0.0002])
        
        # Return object with properties
        return(ee.Feature(geom).set(properties))

    # Create equator feature collection
    out_fc = ee.FeatureCollection(ee.List.sequence(0, in_fc_size.subtract(1), 1).map(pts_to_equator)).set('f_id', 'id')

    # Get ID property name
    prop = out_fc.first().propertyNames().remove('system:index')

    # Reduce ID property to image
    id_i = out_fc.reduceToImage(properties = prop, reducer = ee.Reducer.mean())\
            .rename(['id'])\
            .set("system:index", out_fc.get('f_id'))\
            .set("land-unit", land_unit)\
            .set("in-fc-path", in_fc_path)\
            .set("in-fc-id", in_fc_id)\
            .set("in-ic-path", in_ic_path)\
            .set("in-var-type", in_var_type)\
            .set("var-name", var_name)

    # Return the id image and the points geometry for creating image export geometry
    return(ee.List([id_i, out_fc]))

# Apply ID image function to input feature collection
out_list = generate_id_i(in_fc)
out_i = ee.Image(out_list.get(0))
out_fc = ee.FeatureCollection(out_list.get(1))

# Generate empty Image Collection asset to append images
os.system(f"earthengine create collection {out_path}")

# Export ID image to new Image Collection
task = ee.batch.Export.image.toAsset(
    image = id_i,
    description = f'Initialize - {var_name_exp} - id',
    assetId = out_path + '/id',
    region = out_fc.geometry().buffer(20),
    scale = 22.264,
    maxPixels = 1e13)
task.start()

Asset projects/dri-apps/assets/blm-database/long-term-drought-blend already exists.


## Pre-process data

### Calculate GridMET drought blends

In [75]:
# Define property list
property_list = ["system:index", "system:time_start"]

# Function to calculate short-term and long-term blends
def preprocess_gm_drought(img):

  # Define preliminary variables for short-term blend calculation
  stb_variable = "short_term_drought_blend"
  stb_pdsi_img = img.select("pdsi")
  stb_z_img = img.select("z")
  stb_spi90d_img = img.select("spi90d")
  stb_spi30d_img = img.select("spi30d")

  # Define weights for short-term blend calculation
  stb_pdsi_coef = 0.2
  stb_z_coef = 0.35
  stb_spi90d_coef = 0.25
  stb_spi30d_coef = 0.2

  # Calculate short-term blend
  stblend = stb_pdsi_img.expression(
      "b() * pdsi_coef / 2 + spi90d * spi90d_coef + spi30d * spi30d_coef + z * z_coef / 2",{
          "spi90d": stb_spi90d_img, 
          "spi30d": stb_spi30d_img, 
          "z": stb_z_img, 
          "pdsi_coef": stb_pdsi_coef,
          "spi90d_coef": stb_spi90d_coef, 
          "spi30d_coef": stb_spi30d_coef, 
          "z_coef": stb_z_coef})

  # Define preliminary variables for long-term blend calculation
  ltb_variable = "long_term_drought_blend"
  ltb_pdsi_img = img.select("pdsi")
  ltb_spi180d_img = img.select("spi180d")
  ltb_spi1y_img = img.select("spi1y")
  ltb_spi2y_img = img.select("spi2y")
  ltb_spi5y_img = img.select("spi5y")

  # Define weights for long-term blend calculation
  ltb_pdsi_coef = 0.35
  ltb_spi180d_coef = 0.15
  ltb_spi1y_coef = 0.2
  ltb_spi2y_coef = 0.2
  ltb_spi5y_coef = 0.1

  # Calculate short-term blend
  ltblend = ltb_pdsi_img.expression(
      "b() * pdsi_coef / 2 + spi180d* spi180d_coef + spi1y * spi1y_coef + spi2y * spi2y_coef + spi5y * spi5y_coef",{
          "spi180d": ltb_spi180d_img, 
          "spi1y": ltb_spi1y_img, 
          "spi2y": ltb_spi2y_img, 
          "spi5y": ltb_spi5y_img,
          "spi180d_coef": ltb_spi180d_coef, 
          "spi1y_coef": ltb_spi1y_coef, 
          "spi2y_coef": ltb_spi2y_coef,
          "spi5y_coef": ltb_spi5y_coef, 
          "pdsi_coef": ltb_pdsi_coef})
  
  return ltblend.addBands(stblend).select([0,1], [ltb_variable, stb_variable]).copyProperties(img, property_list)

# Map function to calculate drought blends over the subset years, convert to monthly median images, and convert to multi-band image
# Filter for dates without NAs for the long term blend
in_ic = in_ic.filterDate('1985-01-01', str(date.today()))\
                 .map(preprocess_gm_drought)

# Convert Image Collection to multi-band image
in_i = in_ic.toBands()

# Select variable to serve as input
in_i = in_i.select(['[0-9]{8}_' + var_name])

# Bandnames must be an eight digit character string 'YYYYMMDD'. Annual data will be 'YYYY0101'.
def replace_name(name):
  return ee.String(name).replace(var_name, '').replace('_', '')

# Finish cleaning input image
in_i = in_i.rename(in_i.bandNames().map(replace_name))

# Get list of date strings from image
in_dates = in_i.bandNames().getInfo()
print(in_dates)

['20220105', '20220110', '20220115', '20220120', '20220125', '20220130']


In [66]:
# Iterate over in_dates with functions
in_date = in_dates[0]

# Select date band
in_i_date = in_i.select([in_date])
print(in_i_date.bandNames().getInfo())

['20220105']


## Reduce regions to time-series centroid Feature Collection

In [67]:
# Function to return feature time-series as centroid feature collection for continuous variabless
def img_to_pts_continuous(in_i):
    
    # Cast input image to ee.Image
    img = ee.Image(in_i)
    
    # Get resolution of the image
    res = img.select(0).projection().nominalScale()
    
    # Run reduce regions for allotments and select only the columns with reducers
    img_rr = img.reduceRegions(collection = in_fc, reducer = ee.Reducer.percentile([5, 25, 50, 75, 95])\
                                .combine(reducer2 = ee.Reducer.mean(), sharedInputs = True),\
                                scale = res,\
                                tileScale = 16).select(['mean', 'p.*'])
    
    # Get list of RR features
    img_rr_list = img_rr.toList(img_rr.size())
    
    # Get size of RR features
    img_rr_size = img_rr_list.size()
    
    # Function to create feature collection next to equator
    def pts_to_equator(i):
        
        # Cast number to EE number
        i = ee.Number(i)
        
        # Get properties
        properties = ee.Feature(img_rr_list.get(i)).toDictionary()
        
        # Create geometry at equator
        geom = ee.Geometry.Point([i.multiply(0.0002),0.0002])
        
        # Return object with properties
        return(ee.Feature(geom).set(properties))
    
    # Create equator feature collection
    equator_fc = ee.FeatureCollection(ee.List.sequence(0, img_rr_size.subtract(1), 1).map(pts_to_equator))
    
    return(equator_fc)

In [68]:
# Run function to get time-series statistics for input feature collection for continuous variables
out_fc = img_to_pts_continuous(in_i_date)
print(out_fc.first().propertyNames().sort().getInfo())
print(out_fc.size().getInfo())

['mean', 'p25', 'p5', 'p50', 'p75', 'p95', 'system:index']
148


## Convert centroid time-series to Image Collection time-series

In [69]:
# Function to generate series image collection from feature collections
def pts_to_img_continuous(in_fc):
    
    # Cast to FeatureCollections
    fc = ee.FeatureCollection(in_fc)
    
    # Get list of properties to iterate over for creating multiband image for each date
    props = fc.first().propertyNames().remove('system:index')
    
    # Function to generate image from stats stored in Feature Collection property
    def generate_stat_image(prop):
        img = fc.reduceToImage(properties = [prop], reducer = ee.Reducer.sum()).rename([prop])
        return(img)
    
    # Generate multi-band stats image
    img_mb = ee.ImageCollection(props.map(generate_stat_image)).toBands()\
            .rename(props)\
            .set("system:index", out_fc.get('f_id'))\
            .set("land-unit", land_unit)\
            .set("in-fc-path", in_fc_path)\
            .set("in-fc-id", in_fc_id)\
            .set("in-ic-path", in_ic_path)\
            .set("in-var-type", in_var_type)\
            .set("var-name", var_name)
    
    return(img_mb)

In [70]:
# Convert centroid time-series to image collection time-series
out_i_date = pts_to_img_continuous(out_fc)
print(out_i_date.bandNames().getInfo())

['p25', 'p5', 'mean', 'p50', 'p95', 'p75']


In [31]:
# # Create export geometry
# geometry = rr_fc.geometry().bounds().buffer(1000)

# Map = geemap.Map()
# Map.addLayer(ee.Image(rr_ic.first()))
# Map.addLayer(geometry)
# Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Export Images to Image Collection Asset

In [72]:
# Export ID image to new Image Collection
task = ee.batch.Export.image.toAsset(
    image = out_i_date,
    description = f'Initialize - {var_name_exp} - {in_date}',
    assetId = f'{out_path}/{in_date}',
    region = out_fc.geometry().buffer(20),
    scale = 22.264,
    maxPixels = 1e13)
task.start()

In [76]:
!earthengine rm --recursive projects/ce-datasets/assets/blm-allotment-ics/blm-rap-cover-afg